In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import networkx as nx
import numpy as np
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.model_selection import train_test_split

In [4]:
# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# 1. Environment Setup
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# 2. Preprocessing - Generate random graph and features
num_nodes = 100
num_features = 16
num_classes = 3

G = nx.erdos_renyi_graph(num_nodes, 0.1)
edge_index = torch.tensor(list(G.edges), dtype=torch.long).t().contiguous()
x = torch.randn((num_nodes, num_features))
y = torch.randint(0, num_classes, (num_nodes,))

data = Data(x=x, edge_index=edge_index, y=y)

# 3. Train-Test Split
train_mask, test_mask = train_test_split(np.arange(num_nodes), test_size=0.2, random_state=42)
data.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.test_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.train_mask[train_mask] = True
data.test_mask[test_mask] = True

In [5]:
# 4. Train Base Model (GCN)
model = GCN(in_channels=num_features, hidden_channels=32, out_channels=num_classes)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.NLLLoss()

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

# 5. Planning (Graph Structure Exploration & Embedding Analysis)
def explore_graph():
    print(f"Graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
    degrees = [deg for _, deg in G.degree()]
    print(f"Average node degree: {np.mean(degrees):.2f}")

def analyze_embeddings():
    model.eval()
    with torch.no_grad():
        embeddings = model.conv1(data.x, data.edge_index)
    print(f"Embedding shape: {embeddings.shape}")

# 6. Fine-Tune Model
def fine_tune():
    for param_group in optimizer.param_groups:
        param_group['lr'] = 0.005  # Adjust learning rate

# 7. Evaluate
def evaluate():
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out[data.test_mask].argmax(dim=1)
        acc = (pred == data.y[data.test_mask]).sum().item() / data.test_mask.sum().item()
    print(f"Test Accuracy: {acc:.4f}")

# 8. Deploy Policy (Mock Deployment)
def deploy_policy():
    print("Model ready for deployment.")

# Workflow Execution
explore_graph()
for epoch in range(10):
    loss = train()
    print(f"Epoch {epoch + 1}, Loss: {loss:.4f}")

analyze_embeddings()
fine_tune()
for epoch in range(5):
    loss = train()
    print(f"Fine-Tuning Epoch {epoch + 1}, Loss: {loss:.4f}")

evaluate()
deploy_policy()

Graph has 100 nodes and 529 edges.
Average node degree: 10.58
Epoch 1, Loss: 1.1699
Epoch 2, Loss: 1.0888
Epoch 3, Loss: 1.0625
Epoch 4, Loss: 1.0525
Epoch 5, Loss: 1.0351
Epoch 6, Loss: 1.0124
Epoch 7, Loss: 0.9918
Epoch 8, Loss: 0.9774
Epoch 9, Loss: 0.9689
Epoch 10, Loss: 0.9628
Embedding shape: torch.Size([100, 32])
Fine-Tuning Epoch 1, Loss: 0.9554
Fine-Tuning Epoch 2, Loss: 0.9505
Fine-Tuning Epoch 3, Loss: 0.9445
Fine-Tuning Epoch 4, Loss: 0.9381
Fine-Tuning Epoch 5, Loss: 0.9320
Test Accuracy: 0.2500
Model ready for deployment.
